Rede neural CNN treinado sob dados brutos de forma binária

1º Importar as bibliotecas e os dados do conjunto MNIST

In [ ]:
import json
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models

# 1. Carregar o conjunto de dados MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

2º Criar os filtros

In [ ]:
classes = [0, 1]
filter_train = (y_train == classes[0]) | (y_train == classes[1])
filter_test = (y_test == classes[0]) | (y_test == classes[1])

3º Filtrar os dados

In [ ]:
x_train, y_train = x_train[filter_train], y_train[filter_train]
x_test, y_test = x_test[filter_test], y_test[filter_test]

4º Normalizar os dados

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0

5º Pré-processamento os dados

In [ ]:
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

6º Construir o modelo da CNN

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

7º Compilar o modelo

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

- Criando e salvando os hiperparâmetros da arquitetura e da inicialização em formato json em arquivos-brutos/hiperparametros.json

- Salvando os pesos iniciais

In [ ]:
# Definir os hiperparâmetros da arquitetura da rede neural e os hiperparâmetros de inicialização
hiperparametros = {
    'arquitetura': {
        'camadas': [
            {'tipo': 'Conv2D', 'filtros': 32, 'kernel_size': (3, 3), 'activation': 'relu', 'input_shape': (28, 28, 1)},
            {'tipo': 'MaxPooling2D', 'pool_size': (2, 2)},
            {'tipo': 'Conv2D', 'filtros': 64, 'kernel_size': (3, 3), 'activation': 'relu'},
            {'tipo': 'MaxPooling2D', 'pool_size': (2, 2)},
            {'tipo': 'Conv2D', 'filtros': 64, 'kernel_size': (3, 3), 'activation': 'relu'},
            {'tipo': 'Flatten'},
            {'tipo': 'Dense', 'unidades': 64, 'activation': 'relu'},
            {'tipo': 'Dense', 'unidades': 1, 'activation': 'sigmoid'}
        ]
    },
    'inicializacao': {
        'optimizer': 'adam',
        'loss': 'binary_crossentropy',
        'metrics': ['accuracy']
    }
}

# Serializando os hiperparâmetros em uma string JSON
hiperparametros_json = json.dumps(hiperparametros, indent=4)

# Escrevendo a string JSON em um arquivo
with open("arquivos-brutos/hiperparametros.json", "w") as arquivo:
    arquivo.write(hiperparametros_json)


# PESOS INICIAIS
model.save_weights('arquivos-brutos/pesos_iniciais.weights.h5')

8º Treinar o modelo

In [ ]:
historico = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

- Salvando os pesos finais da rede 
- Salvando o histórico de perda para cada iteração
- Salvando as saídas produzidas pela rede para cada um dos dados de teste

In [ ]:
# PESOS FINAIS
model.save_weights('arquivos-brutos/pesos_finais.weights.h5')

# ERRO DE CADA ITERAÇÃO
perdas = historico.history['loss']

# Salvando o histórico de perda em um arquivo JSON
with open('arquivos-brutos/historico_perda.json', 'w') as f:
    json.dump(perdas, f)


# SAÍDAS PRODUZIDAS
# Fazendo inferência com o modelo treinado para obter as saídas
saidas = model.predict(x_train)

# Convertendo as saídas para uma lista para serialização
saidas_lista = saidas.tolist()

# Salvando as saídas em um arquivo JSON
with open('arquivos-brutos/saidas_teste.json', 'w') as f:
    json.dump(saidas_lista, f)

9º Testar o modelo

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')